In [2]:
import meb
from meb import utils
from meb import datasets
from meb import core
from meb import models

from functools import partial
from typing import List, Tuple

import numpy as np
import pandas as pd
from numba import jit, njit
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import timm
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.multioutput import MultiOutputClassifier



pd.set_option("display.max_columns", 50)
%load_ext autoreload
%autoreload 2

In [3]:
c = datasets.CrossDataset(optical_flow=True, resize=28)
df = c.data_frame
data = c.data

In [4]:
class GeneticAlgorithm:
    def __init__(self, features: np.ndarray, labels: np.ndarray, subject_idx: int,
                 population_size: int = 50, d: int = 50, chr_length: int = 400,
                 change_rate: float = 0.8, multi_label: bool = False
    ):
        self.features = features
        self.labels = labels
        self.subject_idx = subject_idx
        self.population_size = population_size
        self.d = d
        self.chr_length = chr_length
        self.change_rate = change_rate
        self.clf = SVC(kernel="rbf", C=2, random_state=42)
        if multi_label:
            self.clf = MultiOutputClassifier(self.clf)
    
    def perform(self) -> Tuple[np.ndarray, np.ndarray]:
        pop = np.random.randint(2, size=(self.population_size, self.chr_length))
        for i in tqdm(range(self.d)):
            a_pop = crosspop(pop, self.change_rate)
            b_pop = variation(pop)
            new_pop = np.concatenate((pop, a_pop, b_pop))
            _, next_pop = self.select_svm(new_pop)
            pop = next_pop[:self.population_size]
            best_pop = next_pop[0]
        prediction = self.evaluate(best_pop)
        return prediction
    
    def select_svm(self, pop: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
        d, w = pop.shape
        AC = np.zeros((self.d, 2))
        for i in range(self.d):
            if sum(pop[i]) != 0:
                # Select features based on the mutation
                selected_features = self.features[:, pop[i] == 1]
                # Split the data to train and test
                train_features = selected_features[~self.subject_idx]
                train_labels = self.labels[~self.subject_idx]
                # Cross validation with subjects not in the testing data
                scores = cross_val_score(self.clf, train_features, train_labels, cv=5)
                AC[i] = [np.mean(scores), i]
            else:
                AC[i] = [0, i]
        # Sort based on best performance
        idx = np.argsort(AC[:, 0])[::-1]
        AC = AC[idx]
        new_pop = pop[idx]
        return AC, new_pop  

    def evaluate(self, pop: np.ndarray) -> np.ndarray:
        # Select features based on the mutation
        selected_features = self.features[:, pop == 1]
        # Split the data to train and test
        train_features = selected_features[~self.subject_idx]
        train_labels = self.labels[~self.subject_idx]
        test_features = selected_features[self.subject_idx]
        test_labels = self.labels[self.subject_idx]
        # Train and evaluate
        self.clf.fit(train_features, train_labels)
        prediction = self.clf.predict(test_features)
        return prediction
    
    
@njit
def judgepop(pop1, pop2):
    return (pop1 != pop2).sum()


@njit
def concat(arr1, arr2) -> np.ndarray:
    """Custom concat function to avoid case where one array
    of np.concatenate is empty which numba doesn't like"""
    if len(arr1) == 0:
        return arr2
    elif len(arr2) == 0:
        return arr1
    return np.concatenate((arr1, arr2))


@njit(fastmath=True)
def variation(pop: np.ndarray) -> np.ndarray:
    rate = 0.2
    u, w = pop.shape
    n_pop = np.zeros_like(pop)
    for i in range(u):
        if i == 0 or judgepop(pop[0], pop[i]) >= w * rate:
            locate = int(np.random.rand() * w)
            n_pop[i] = pop[i]
            if pop[i, locate] == 0:
                n_pop[i, locate] = 1
            else:
                n_pop[i, locate] = 0
        else:
            x = np.round(w * rate)
            temp = np.random.permutation(w)[:x]
            n_pop[i] = pop[i]
            for j in range(x):
                if pop[i, temp[j]] == 0:
                    n_pop[i, temp[j]] = 1
                else:
                    n_pop[i, temp[j]] = 0
    return n_pop


@njit
def crosspop(pop: np.ndarray, change_rate: float) -> np.ndarray:
    u, w = pop.shape
    m = np.random.permutation(u)
    n_pop = np.zeros_like(pop)
    for i in range(0, u, 2):
        if np.random.rand(1) < change_rate:
            locate = np.int(np.random.rand() * w)
            n_pop[i] = concat(pop[m[i], :locate], pop[m[i + 1], locate:])
            n_pop[i + 1] = concat(pop[m[i + 1], :locate], pop[m[i], locate:]) 
    return n_pop

In [5]:
from torch import optim
from torch import nn
from sklearn import svm
from sklearn.model_selection import cross_val_score

In [6]:
class STSTNet(nn.Module):
    def __init__(self, in_channels: int = 3, num_channels: int = 3, **kwargs):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels=3, kernel_size=3, padding=2)
        self.conv2 = nn.Conv2d(in_channels, out_channels=5, kernel_size=3, padding=2)
        self.conv3 = nn.Conv2d(in_channels, out_channels=8, kernel_size=3, padding=2)
        self.relu = nn.ReLU()
        self.bn1 = nn.BatchNorm2d(3)
        self.bn2 = nn.BatchNorm2d(5)
        self.bn3 = nn.BatchNorm2d(8)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=3, padding=1)
        self.avgpool = nn.AvgPool2d(kernel_size=2, stride=2, padding=0)
        self.dropout = nn.Dropout(p=0.5)
        self.fc = nn.Linear(in_features=5 * 5 * 16, out_features=num_channels)

    def forward(self, x, only_features: bool = False):
        x1 = self.dropout(self.maxpool(self.bn1(self.relu(self.conv1(x)))))
        x2 = self.dropout(self.maxpool(self.bn2(self.relu(self.conv2(x)))))
        x3 = self.dropout(self.maxpool(self.bn3(self.relu(self.conv3(x)))))
        x = torch.cat((x1, x2, x3), 1)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        if only_features:
            return x
        x = self.fc(x)
        return x

In [7]:
class Config(core.Config):
    action_units = utils.dataset_aus["cross"]
    device = torch.device("cuda:1")
    epochs = 500
    num_workers = 0
    optimizer = partial(optim.Adam, lr=5e-5, weight_decay=1e-3)
    scheduler = None
    model = partial(STSTNet, num_channels=len(action_units))
    game_d = 50
    evaluation_fn = [
        partial(utils.MultiLabelF1Score, average="macro"),
        partial(utils.MultiLabelF1Score, average="binary"),
    ]

In [10]:
out = GAMEValidation(Config).validate_n_times(df, data, n_times=5)

100%|████████████████████████████████████████| 50/50 [5:32:58<00:00, 399.57s/it]

100%|████████████████████████████████████████| 50/50 [5:05:47<00:00, 366.96s/it]

100%|████████████████████████████████████████| 50/50 [4:41:50<00:00, 338.22s/it]

 78%|███████████████████████████████▏        | 39/50 [3:24:46<58:10, 317.33s/it]


100%|████████████████████████████████████████| 50/50 [4:23:40<00:00, 316.41s/it]

100%|████████████████████████████████████████| 50/50 [4:31:33<00:00, 325.87s/it]

100%|████████████████████████████████████████| 50/50 [4:43:54<00:00, 340.68s/it]

 56%|█████████████████████▎                | 28/50 [2:29:05<1:56:49, 318.62s/it]


100%|████████████████████████████████████████| 50/50 [4:26:04<00:00, 319.29s/it]

100%|████████████████████████████████████████| 50/50 [4:54:49<00:00, 353.80s/it]

100%|████████████████████████████████████████| 50/50 [4:16:57<00:00, 308.35s/it]

100%|████████████████████████████████████████| 50/50 [4:02:31<00:00, 291.04s/it]

 36%|██████████████▍                         | 18/50 [51:19<1:31:02, 170.71s/it]


100%|████████████████████████████████████████| 50/50 [4:32:52<00:00, 327.46s/it]

100%|████████████████████████████████████████| 50/50 [4:24:50<00:00, 317.81s/it]

100%|████████████████████████████████████████| 50/50 [4:47:55<00:00, 345.52s/it]

 14%|█████▋                                   | 7/50 [35:38<3:38:42, 305.18s/it]


100%|████████████████████████████████████████| 50/50 [4:15:45<00:00, 306.92s/it]

100%|████████████████████████████████████████| 50/50 [3:56:50<00:00, 284.21s/it]

 94%|█████████████████████████████████████▌  | 47/50 [4:26:35<17:08, 342.95s/it]


100%|████████████████████████████████████████| 50/50 [4:43:37<00:00, 340.36s/it]

100%|████████████████████████████████████████| 50/50 [4:27:09<00:00, 320.60s/it]

100%|████████████████████████████████████████| 50/50 [4:49:49<00:00, 347.79s/it]

100%|████████████████████████████████████████| 50/50 [4:02:56<00:00, 291.54s/it]

 74%|████████████████████████████          | 37/50 [2:54:14<1:01:09, 282.23s/it]


100%|████████████████████████████████████████| 50/50 [3:55:01<00:00, 282.02s/it]

100%|████████████████████████████████████████| 50/50 [4:21:06<00:00, 313.32s/it]

100%|████████████████████████████████████████| 50/50 [4:14:43<00:00, 305.66s/it]

 52%|███████████████████▊                  | 26/50 [2:26:03<2:14:59, 337.47s/it]


100%|████████████████████████████████████████| 50/50 [4:41:02<00:00, 337.25s/it]

100%|████████████████████████████████████████| 50/50 [4:11:06<00:00, 301.33s/it]

100%|████████████████████████████████████████| 50/50 [3:49:56<00:00, 275.92s/it]

100%|██████████████████████████████████████| 5/5 [125:54:42<00:00, 90656.56s/it]


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
83.3 & 81.6 & 89.3 & 48.7 & 49.2 & 59.3 & 52.2 & 50.1 & 54.0 & 53.8 & 49.3 & 66.3 & 61.4

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
59.6 & 63.6 & 63.9 & 59.1 & 62.3 & 60.9 & 61.6
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
71.1 & 67.6 & 85.7 & 1.1 & 0.0 & 26.2 & 7.8 & 2.1 & 12.8 & 15.4 & 0.0 & 34.7 & 27.0

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
23.8 & 31.0 & 32.0 & 23.3 & 29.6 & 25.6 & 27.6


MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
83.3 & 81.6 & 89.3 & 48.7 & 49.2 & 59.3 & 52.2 & 50.1 & 54.0 & 53.8 & 49.3 & 66.3 & 61.4

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
59.6 & 63.6 & 63.9 & 59.1 & 62.3 & 60.9 & 61.6
MultiLabelF1Score
AUS: ['AU1', 'AU2', 'AU4', 'AU5', 'AU6', 'AU7', 'AU9', 'AU10', 'AU12', 'AU14', 'AU15', 'AU17', 'Average']
71.1 & 67.6 & 85.7 & 1.1 & 0.0 & 26.2 & 7.8 & 2.1 & 12.8 & 15.4 & 0.0 & 34.7 & 27.0

Datasets:  ['casme', 'casme2', 'samm', 'fourd', 'mmew', 'casme3a', 'Average']
23.8 & 31.0 & 32.0 & 23.3 & 29.6 & 25.6 & 27.6

In [9]:
class GAMEValidation(core.CrossDatasetValidation):
    def __init__(self, config: Config, verbose: bool = True):
        super().__init__(config)
        self.verbose = verbose
        self.disable_tqdm = True

    def extract_features(self, dataloader: torch.utils.data.DataLoader):
        self.model.eval()
        features_list = []
        for batch in dataloader:
            data_batch = batch[0].to(self.cf.device)
            labels_batch = batch[1]
            features = self.model(data_batch.float(), only_features=True)
            features_list.append(features.detach().cpu())
        self.model.train()
        features = torch.cat(features_list)
        return features
    
    def validate_split(self, df: pd.DataFrame, input_data: np.ndarray, labels: np.ndarray, split_name: str):
        train_data, train_labels, test_data, test_labels = self.split_data(
            df[self.split_column], input_data, labels, split_name
        )
        train_loader = self.get_data_loader(train_data, train_labels, train=True)
        test_loader = self.get_data_loader(test_data, test_labels, train=False)
        self.model = self.cf.model()
        self.model.to(self.cf.device)
        self.criterion = self.cf.criterion()
        self.optimizer = self.cf.optimizer(self.model.parameters())
        self.scheduler = self.cf.scheduler(self.optimizer) if self.cf.scheduler else None
        self.mixup_fn = self.cf.mixup_fn() if self.cf.mixup_fn else None
        
        self.train_model(train_loader, test_loader)
        # Create dataloader from all data and extract features
        data_loader = self.get_data_loader(input_data, labels, train=False)
        features = self.extract_features(data_loader)
        #Classify with genetic algorithm
        ga = GeneticAlgorithm(features, labels, df[self.split_column] == split_name, multi_label=True,
                              population_size=self.cf.game_d, d=self.cf.game_d)
        prediction = ga.perform()
        test_metrics = self.evaluation_fn(torch.tensor(labels[df[self.split_column] == split_name]),
                                     torch.tensor(prediction))
        return [], test_metrics, torch.tensor(prediction)